# Star Wars opening text crawl effect
Mimic of Star Wars' opening title text crawl. A text with a (false) <br>
perspective effect goes towards the end of a chosen image,<br>
for example, a background made of stars. Slight fading effect on the text.

Make sure you have ImageMagick-7.0.8-Q16 dowloaded https://imagemagick.org/index.php<br>
and ffmpeg "pip installed".

Start by importing libraries. MoviePy version is 1.0.0 https://zulko.github.io/moviepy/


In [1]:
import numpy as np
from skimage import transform as tf

from moviepy.editor import *
from moviepy.video.tools.drawing import color_gradient

Choose resolution. Here, width 720 and height 405, i.e. width times 9/16 to make it in widescreen mode.

In [2]:
# RESOLUTION 

w = 720
h = w*9//16 
moviesize = w,h

Type in your text. Add blanks before and after text. Linebreaks inserted using commas.

Here I have two texts; the first one from Star Wars, the second one from Blade Runner (commented out in the code below). It's the a Nexus-6 combat model replicant  Roy Batty's monologue, a.k.a. "Tears in rain".

In [3]:
#INSERT THE RAW TEXT (here: "A New Hope")

txt = "\n".join([
"It is a period of civil war. Rebel",
"spaceships, striking from a",
"hidden base, have won their",
"first victory against the evil",
"Galactic Empire. During the battle,",
"Rebel spies managed to steal secret",
"plans to the Empire’s ultimate weapon,",
"the DEATH STAR, an armored space station",
"with enough power to destroy an entire planet."
"",
"",
"",
"",
"",
"",
"",
"",
"",
])

# Add blanks before and after text
txt = 10*"\n" +txt + 10*"\n"

# INSERT THE RAW TEXT (here: Tears in rain -monologue, 
# from Bladerunner)
# txt = "\n".join([
# "I’ve been Offworld and back…",
# "I’ve stood on the back deck of a blinker",
# "bound for the Plutition Camps",
# "with sweat in my eyes.",
# "I’ve known adventures, seen things",
# "you people wouldn't believe.",
# "Attack ships on fire",
# "off the shoulder of Orion.", 
# "I watched C-beams glitter in the dark",
# "near the Tannhäuser Gate.",
# "All those moments",
# "will be lost in time,",
# "like tears in rain.",
# "Time to die...",
# "",
# "",
# "",
# "",
# "",
# "",
# "",
# "",
# "",
# ])
# Add blanks before and after text
#txt = 10*"\n" +txt + 10*"\n"

Apply features for the the text; color, alignment, font size etc. and create the text image.<br> You can check the available fonts and colors in TextClip using the .list -function:<br>
*TextClip.list('font')<br>
TextClip.list('color')*

In [4]:
# CREATE THE TEXT IMAGE

clip_txt = TextClip(txt,color='cyan', align='Center',fontsize=25,
                    font='Xolonium-Bold', method='label')

Set speed and crop the area for the rolling text

In [5]:
# SCROLL THE TEXT IMAGE BY CROPPING A MOVING AREA

txt_speed = 10 
fl = lambda gf,t : gf(t)[int(txt_speed*t):int(txt_speed*t)+h,:]
moving_txt= clip_txt.fl(fl, apply_to=['mask'])

Add a vanishing effect on the text with a gradient, backward leaning mask.

In [6]:
# ADD A VANISHING EFFECT ON THE TEXT WITH A GRADIENT MASK

grad = color_gradient(moving_txt.size,p1=(0,2*h/3),
                p2=(0,h/4),col1=0.0,col2=1.0)
gradmask = ImageClip(grad,ismask=True)
fl = lambda pic : np.minimum(pic,gradmask.img)
moving_txt.mask = moving_txt.mask.fl_image(fl)

To create that perspective effect, warp the text into a trapezoid.

In [7]:
# WARP THE TEXT INTO A TRAPEZOID (PERSPECTIVE EFFECT)

def trapzWarp(pic,cx,cy,ismask=False):
    Y,X = pic.shape[:2]
    src = np.array([[0,0],[X,0],[X,Y],[0,Y]])
    dst = np.array([[cx*X,cy*Y],[(1-cx)*X,cy*Y],[X,Y],[0,Y]])
    tform = tf.ProjectiveTransform()
    tform.estimate(src,dst)
    im = tf.warp(pic, tform.inverse, output_shape=(Y,X))
    return im if ismask else (im*255).astype('uint8')

fl_im = lambda pic : trapzWarp(pic,0.2,0.3)
fl_mask = lambda pic : trapzWarp(pic,0.2,0.3, ismask=True)
warped_txt= moving_txt.fl_image(fl_im)
warped_txt.mask = warped_txt.mask.fl_image(fl_mask)

Add a background image of your choice and darken it if necessary to increase the contrast with the text. Choose a color image (RBG), not black-and-white.

In [8]:
# BACKGROUND IMAGE, DARKENED HERE AT 60%

stars = ImageClip('C:/Users/jangn/OneDrive/CODE/Deep_Learning/Star_Wars_opening_title/images/death_star.jpg')
stars_darkened = stars.fl_image(lambda pic: (0.6*pic).astype('int16'))  

Finally, compose the movie by setting its duration in seconds, fps rate and write it to a file, giving it a name of your choice. 

In [10]:
# COMPOSE THE MOVIE

final = CompositeVideoClip([
         stars_darkened,
         warped_txt.set_pos(('center','bottom'))],
         size = moviesize)


# WRITE TO A FILE
# duration here set to 35 seconds, with 18 frames per second

final.set_duration(35).write_videofile("star_wars_crawl.avi", 
                                       fps=18, codec='rawvideo') 



Moviepy - Building video star_wars_crawl.avi.
Moviepy - Writing video star_wars_crawl.avi



Moviepy - Done !
Moviepy - video ready star_wars_crawl.avi


If you are a Windows 10 user, like me, you most likely will need to do an adjustment in the **moviepy/config_defaults.py** -file. You need to provide the path to the ImageMagick binary called *magick*. You should find the path within the config_defaults.py -file looking like this:

IMAGEMAGICK_BINARY = "C:\\Program Files\\ImageMagick_VERSION\\magick.exe"

Please note! In some versions the file might be called convert.exe instead of magick.exe.

The size of final movie file with the settings above will be about 540Mbs. Feel free to experiment with other settings!